### Distill step by step finetuning approach

In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
import transformers
transformers.__version__

'4.34.1'

In [3]:
#!sudo pip install -q accelerate peft==0.4.0 bitsandbytes trl==0.4.7

In [4]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

In [5]:
df = pd.read_csv('/data/mistral/query-to-mql/training-data.csv')

In [6]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [7]:
df.shape

(37, 3)

In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
#new_model = "mistral-ft-peft-on-template_and_user_query-data"

In [9]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [10]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [11]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/data/mistral/query-to-mql/oct-27"

# Number of training epochs
num_train_epochs = 15

# Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = False
fp16 = True # not using quantisation
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 200

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 20

# Log every X updates steps
logging_steps = 20

In [12]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [13]:
promt_template_measure = """<s>[INST]<<SYS>>
Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.

<</SYS>>
[/INST]
[MQL]
{mql}
[/MQL]

the steps and rationale used to achieve above structured output is as below.
{rationale}
</s>
"""

In [14]:
promt_template = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
{mql}
[/MQL]
the steps and rationale used to achieve above structured output is as below.
{rationale}
"""

In [15]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [13]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [14]:
date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [18]:
def create_fine_tuning_dataset(row):
    mql = row['MQL']
    user_query = row['Query']
    rationale = row['Rationale']
    formated = promt_template.format(context=context,
                                             date_input=date_input,
                                             user_query=user_query,
                                             mql=mql,
                                             rationale=rationale)
    return formated

In [19]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [20]:
df.drop(columns=['Query', 'MQL', 'Rationale'], inplace=True)
df.shape

(37, 1)

In [21]:
train_dataset = Dataset.from_pandas(df)

In [22]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 37
})

In [23]:
train_dataset['fine_tuning_dataset'][0]

'Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"

In [24]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [25]:
bnb_4bit_quant_type

'nf4'

In [26]:
compute_dtype

torch.float16

In [27]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [15]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [12]:
!sudo pip install -q pynvml

In [13]:
from pynvml.smi import nvidia_smi
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 16384.0,
    'free': 15972.9375,
    'unit': 'MiB'}}]}

In [14]:
#!df -H

In [15]:
torch.cuda.is_available()

True

In [33]:
# del model
# torch.cuda.empty_cache()

In [34]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [36]:
max([len(tokenizer.encode(df['fine_tuning_dataset'][i])) for i in range(df.shape[0])])

1576

In [37]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules = ["q_proj", "v_proj"],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [38]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
#     eval_steps=50, # requires when eval_dataset is defined
#     per_device_eval_batch_size=1, # Batch size for evaluation
#     evaluation_strategy="steps", # requires when eval_dataset is defined
    logging_strategy="steps",
    logging_steps=5,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=600,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
#     load_best_model_at_end=True,
#     save_total_limit=1,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False
)

In [39]:
## Getting FLOPs of model

model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 2048)
      )
    }
  )
  * training_arguments.gradient_accumulation_steps
)

#print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

Memory footprint 4.551360512 GB
Flops 87375.791259648 GFLOPs


In [40]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 37
})

In [41]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [42]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.8/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [43]:
# Train model
trainer.train()

Step,Training Loss
5,0.893300
10,0.753500
15,0.552700
20,0.306200
25,0.216700
30,0.161100
35,0.164300
40,0.132500
45,0.128500
50,0.101700


TrainOutput(global_step=600, training_loss=0.05151911248763402, metrics={'train_runtime': 4108.884, 'train_samples_per_second': 0.146, 'train_steps_per_second': 0.146, 'total_flos': 3.411337635970253e+16, 'train_loss': 0.05151911248763402, 'epoch': 16.22})

In [ ]:
# Fine-tuned model name
new_model_name = "mistral-ft-peft-v1-lr-64-with-more-data"

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [59]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj)

In [17]:
from peft import PeftModel, PeftConfig

In [18]:
new_model_name = "/data/mistral/query-to-mql/oct-27/checkpoint-600"

In [45]:
del model
# del trainer
torch.cuda.empty_cache()

In [46]:
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 16384.0,
    'free': 9456.9375,
    'unit': 'MiB'}}]}

In [19]:
# del model
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [22]:
#model.to('cuda')

In [21]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model.generate(input_ids=_inputs.to('cuda'), max_length= 1600, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output
#     return output

In [22]:
%%time
user_query = 'show me the bottom 10 segments basis sales'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  show me the bottom 10 segments basis sales


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


CPU times: user 30.4 s, sys: 1.09 s, total: 31.5 s
Wall time: 31.5 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '10'}]}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [23]:
%%time
user_query = 'top 2 and bottom 3 segments by sales'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  top 2 and bottom 3 segments by sales
CPU times: user 38.9 s, sys: 1.1 s, total: 40 s
Wall time: 40 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'},
     {'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '3'}]}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [24]:
%%time
user_query = 'top 2 segments and bottom 3 sub-category basis quantity'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  top 2 segments and bottom 3 sub-category basis quantity
CPU times: user 42.2 s, sys: 1.08 s, total: 43.2 s
Wall time: 43.3 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}],
  'sub-category': [{'ENTITY': 'Sub-Category',
    'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '3'}]}]},
 'MEASURE': {'quantity': [{'ENTITY': 'Quantity'}]}}

In [25]:
%%time
user_query = 'quantity across segments except consumer and corporate in dubai'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  quantity across segments except consumer and corporate in dubai
CPU times: user 50.7 s, sys: 1.04 s, total: 51.8 s
Wall time: 51.8 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]},
 'FILTER': {'consumer': [{'ENTITY': 'Consumer',
    'EXCLUDE': 'True',
    'PARENT': 'Segment'}],
  'corporate': [{'ENTITY': 'Corporate',
    'EXCLUDE': 'True',
    'PARENT': 'Segment'}],
  'dubai': [{'ENTITY': 'Dubai', 'EXCLUDE': 'True', 'PARENT': 'Country'}]},
 'MEASURE': {'quantity': [{'ENTITY': 'Quantity'}]}}

In [26]:
%%time
user_query = 'in corporate share of phone and binder basis discount'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  in corporate share of phone and binder basis discount
CPU times: user 48.7 s, sys: 1.01 s, total: 49.7 s
Wall time: 49.7 s


{'DIMENSION': {'corporate': [{'ENTITY': 'Corporate',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}],
  'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [31]:
%%time
user_query = 'list of under performing segments'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  list of under performing segments
CPU times: user 36.6 s, sys: 1.08 s, total: 37.7 s
Wall time: 37.7 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '1'}]}]},
 'MEASURE': {'under performing': [{'ENTITY': 'Purchase Vol',
    'THRESHOLD': [{'COMPARISON VALUE': 'low',
      'COMPARSION OPERATOR': 'than',
      'VALUE': '100'}]}]}}

In [32]:
raw

'<s> Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile pho

In [28]:
%%time
user_query = 'when was the first time sales of segments was 0'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  when was the first time sales of segments was 0
CPU times: user 49.8 s, sys: 1.12 s, total: 51 s
Wall time: 51 s


{'DATE VARIABLE': {'when': [{'CONVERTED TIME ELEMENT': 'when',
    'ENTITY': 'Order Date'}]},
 'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '0',
      'COMPARSION OPERATOR': '='}]}]}}

In [29]:
%%time
user_query = 'sales of segments from beginning'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales of segments from beginning
CPU times: user 43.2 s, sys: 1.04 s, total: 44.2 s
Wall time: 44.2 s


{'DATE VARIABLE': {'beginning': [{'CONVERTED TIME ELEMENT': 'beginning',
    'DATE RANGE': '01/01/2020 - 30/09/2023',
    'ENTITY': 'Order Date'}]},
 'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [30]:
raw

'<s> Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile pho

In [33]:
%%time
user_query = 'sales in 20/01/2020'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales in 20/01/2020
CPU times: user 39.9 s, sys: 1e+03 ms, total: 40.9 s
Wall time: 40.9 s


{'DATE VARIABLE': {'20/01/2020': [{'CONVERTED TIME ELEMENT': '20 January 2020',
    'DATE RANGE': '2020/01/20 - 2020/01/20',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [34]:
raw

'<s> Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile pho

In [35]:
%%time
user_query = 'sales in last one and half years'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales in last one and half years
CPU times: user 38.1 s, sys: 1.07 s, total: 39.2 s
Wall time: 39.1 s


{'DATE VARIABLE': {'last one and half years': [{'CONVERTED TIME ELEMENT': 'last one and half years',
    'DATE RANGE': '2019/04/01 - 2023/09/30',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [37]:
raw.split('\nStep 1:')[1]

' Identify the measure\nThe query asks for "sales," which is a measure. From the context, we can map "sales" to the "Sales" entity.\n\nStep 2: Identify the date variable and time frame\nThe query asks for "last one and half years," which is a time frame. From the context, we can map this to the "Order Date" entity. To calculate the date range for "last one and half years," we subtract 1.5 years from the end_date (15/09/2023) provided in the date reference. This gives us a date range of 2019/04/01 - 2023/09/30.\n\nStep 3: Create the structured output\nNow that we have identified the measure and the date variable with its time frame, we can create the structured output. The output will have two main keys: "MEASURE" and "DATE VARIABLE." Under "MEASURE," we will have "sales" mapped to the "Sales" entity. Under "DATE VARIABLE," we will have "last one and half years" mapped to the "Order Date" entity with the calculated date range.\n\nRationale:\nThe query is asking for sales data in the las

In [38]:
%%time
user_query = 'trend of sales in dubai now'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  trend of sales in dubai now
CPU times: user 36.5 s, sys: 1.02 s, total: 37.5 s
Wall time: 37.5 s


{'DATE VARIABLE': {'trend': [{'CONVERTED TIME ELEMENT': 'now',
    'DATE RANGE': '2023/09/30 - 2023/09/30',
    'ENTITY': 'Order Date'}]},
 'FILTER': {'dubai': [{'ENTITY': 'Dubai', 'PARENT': 'Country'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [39]:
raw.split('\nStep 1:')[1]

' Identify the main components of the query\nThe query asks for the "trend of sales" in a specific location, "Dubai". \n\nStep 2: Map the components to the context\n- "sales" can be mapped to the "Sales" entity under the \'MEASURE\' category.\n- "Dubai" can be mapped to the "Dubai" entity under the \'FILTER\' category.\n\nStep 3: Identify the date variable\nThe query does not specify a date, so we use the date reference provided, which is \'15/09/2023\'. We can represent this as "now" in the structured output.\n\nStep 4: Create the structured output\n- For the "DATE VARIABLE", we include "now" as the converted time element and provide the date range as "2023/09/30 - 2023/09/30" (which corresponds to the end of the date reference period). We also specify the "ENTITY" as "Order Date".\n- For the "MEASURE", we include "Sales" as the entity.\n\nRationale:\nThe structured output is created by breaking down the query into its main components and mapping them to the provided context. This all

In [40]:
%%time
user_query = 'sales in the week of 01 March 2022'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales in the week of 01 March 2022
CPU times: user 34.6 s, sys: 1.07 s, total: 35.6 s
Wall time: 35.6 s


{'DATE VARIABLE': {'week of 01 March 2022': [{'CONVERTED TIME ELEMENT': 'week of 1 March 2022',
    'DATE RANGE': '2022/02/28 - 2022/03/06',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [41]:
raw.split('\nStep 1:')[1]

' Identify the measure\nThe query asks for "sales," which is a measure. From the context, we can map "sales" to the "Sales" entity.\n\nStep 2: Identify the date variable\nThe query asks for "the week of 01 March 2022," which is a date reference. From the context, we can map this to the "Order Date" entity.\n\nStep 3: Convert the date reference\nThe date reference "week of 01 March 2022" needs to be converted into a date range. Since it\'s a week, we calculate the date range as "2022/02/28 - 2022/03/06."\n\nStep 4: Create the structured output\nNow that we have identified the measure and date variable, and converted the date reference, we can create the structured output. We include the "Sales" entity under the "MEASURE" key and the converted date range under the "DATE VARIABLE" key, along with the associated "Order Date" entity.\n\nRationale:\nThe structured output is created by identifying the key components of the query (measure and date variable) and mapping them to the relevant ent

In [42]:
%%time
user_query = 'top most selling segments in 2020 vs 2021'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  top most selling segments in 2020 vs 2021
CPU times: user 42.4 s, sys: 1.06 s, total: 43.5 s
Wall time: 43.5 s


{'DATE VARIABLE': {'2020 vs 2021': [{'CONVERTED TIME ELEMENT': '2020 vs 2021',
    'DATE RANGE': '2020/01/01 - 2020/12/31 | 2021/01/01 - 2021/12/31',
    'ENTITY': 'Order Date'}]},
 'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '1'}]}]},
 'MEASURE': {'selling': [{'ENTITY': 'Sales'}]}}

In [43]:
%%time
user_query = 'sales from 100 to 200'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales from 100 to 200
CPU times: user 39.4 s, sys: 1.05 s, total: 40.5 s
Wall time: 40.5 s


{'MEASURE': {'sales': [{'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '100',
      'COMPARSION OPERATOR': '>='},
     {'COMPARISON VALUE': '200', 'COMPARSION OPERATOR': '<='}]}]}}

In [44]:
raw.split('\nStep 1:')[1]

' Identify the measure in the query\nThe query mentions "sales," which corresponds to the "Sales" entity in the context.\n\nStep 2: Identify the constraints in the query\nThe query specifies a range for sales: "from 100 to 200." This indicates a constraint on the "Sales" measure, with a lower bound of 100 and an upper bound of 200.\n\nStep 3: Map the constraints to comparison operators\nThe lower bound "from 100" can be represented as a comparison operator ">=" (greater than or equal to) with a comparison value of "100." The upper bound "to 200" can be represented as a comparison operator "<=" (less than or equal to) with a comparison value of "200."\n\nStep 4: Structure the output\nThe structured output should include the identified measure ("Sales") and the corresponding constraints with their comparison operators and values. This is represented in the "MEASURE" section of the output, with the "Sales" entity and the "MEASURE CONSTRAINT" array containing the two constraints with their

In [45]:
%%time
user_query = 'sales from 2021 to 2023'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales from 2021 to 2023
CPU times: user 40.6 s, sys: 1.01 s, total: 41.6 s
Wall time: 41.6 s


{'DATE VARIABLE': {'2021 to 2023': [{'CONVERTED TIME ELEMENT': '2021 to 2023',
    'DATE RANGE': '2021/01/01 - 2023/12/31',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [46]:
raw.split('\nStep 1:')[1]

' Identify the measure\nThe query asks for "sales," which is a measure. From the context, we can map "sales" to the "Sales" entity.\n\nStep 2: Identify the date range\nThe query asks for sales data "from 2021 to 2023." This is a date range. From the context, we can map "2021 to 2023" to the "Order Date" entity.\n\nStep 3: Convert the date range\nConvert the date range "2021 to 2023" into a format that can be used by the system. This can be done by specifying the start and end dates, which are "2021/01/01" and "2023/12/31," respectively.\n\nStep 4: Create the structured output\nCreate a structured output that includes the identified measure and date range. This can be done by creating a "MEASURE" key with the "Sales" entity and a "DATE VARIABLE" key with the "Order Date" entity and the converted date range "2021 to 2023."\n\nRationale:\nThe structured output is created by identifying the measure and date range from the query and converting the date range into a format that can be used b

In [47]:
%%time
user_query = 'what is purchase across segments'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  what is purchase across segments
CPU times: user 33 s, sys: 1.05 s, total: 34 s
Wall time: 34 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [48]:
raw.split('\nStep 1:')[1]

' Identify the main components of the query\nIn the given query "what is purchase across segments", the main components are "purchase" and "segments".\n\nStep 2: Map the components to the context\n- "purchase" can be mapped to the \'Purchase Vol\' entity under \'MEASURE\' in the context.\n- "segments" can be mapped to the \'Segment\' entity under \'DIMENSION\' in the context.\n\nStep 3: Create the structured output\n- For the \'DIMENSION\' part of the output, we include the \'Segment\' entity since the query asks for purchase across segments.\n- For the \'MEASURE\' part of the output, we include the \'Purchase Vol\' entity as the query is asking for the purchase value.\n\nRationale:\nThe structured output is created by identifying the main components of the query and mapping them to the given context. This helps in understanding the user\'s intent and provides a structured format that can be used for further analysis or processing.\n\nBy following these steps, the query "what is purcha

In [49]:
%%time
user_query = 'discount rate of phone and binders'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  discount rate of phone and binders
CPU times: user 50.2 s, sys: 995 ms, total: 51.2 s
Wall time: 51.2 s


{'DIMENSION': {'binders': [{'ENTITY': 'Binder',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}],
  'phone': [{'ENTITY': 'Phone',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'binders': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}],
  'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'discount rate': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [52]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- In the given query "discount rate of phone and binders", the main components are "discount rate", "phone", and "binders".

Step 2: Map the components to the context
- "discount rate" can be mapped to the "Discount" entity under the "MEASURE" category.
- "phone" can be mapped to the "Phone" entity under the "FILTER" category with "Sub-Category" as its parent.
- "binders" can be mapped to the "Binder" entity under the "FILTER" category with "Sub-Category" as its parent.

Step 3: Create the structured output
- For the "DIMENSION" part, include the "Binder" and "Phone" entities with their respective ranks (if any).
- For the "FILTER" part, include the "Binder" and "Phone" entities with their respective parents ("Sub-Category").
- For the "MEASURE" part, include the "Discount" entity with any measure constraints (if any).

Step 4: Combine the structured output components
- Combine the "DIMENSION", "FILTER", and "MEASURE" components to form

In [53]:
%%time
user_query = 'discount rate of overall sub-category in corporate'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  discount rate of overall sub-category in corporate
CPU times: user 39.8 s, sys: 1.01 s, total: 40.8 s
Wall time: 40.8 s


{'DIMENSION': {'sub-category': [{'ENTITY': 'Sub-Category',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}]},
 'MEASURE': {'discount rate': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [54]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "discount rate of overall sub-category in corporate", the main components are "discount rate", "sub-category", and "corporate".

Step 2: Map the components to the context
- "discount rate" can be mapped to the 'Discount' entity under 'MEASURE'.
- "sub-category" can be mapped to the 'Sub-Category' entity under 'DIMENSION'.
- "corporate" can be mapped to the 'Corporate' entity under 'FILTER' with 'Segment' as its parent.

Step 3: Create the structured output
- For the 'DIMENSION' part, include the 'Sub-Category' entity with no rank specified.
- For the 'FILTER' part, include the 'Corporate' entity with 'Segment' as its parent.
- For the 'MEASURE' part, include the 'Discount' entity with no measure constraint specified.

Rationale:
The query asks for the discount rate of overall sub-categories in the corporate segment. To answer this, we need to consider the 'Sub-Category' dimension, apply the 'Corporate' filter, and cal

In [55]:
%%time
user_query = 'maximum sales of phone for consumer segment'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  maximum sales of phone for consumer segment
CPU times: user 44.4 s, sys: 1.02 s, total: 45.5 s
Wall time: 45.5 s


{'DIMENSION': {'segment': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'consumer': [{'ENTITY': 'Consumer', 'PARENT': 'Segment'}],
  'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'sales': [{'ADJECTIVE': ['maximum'],
    'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}],
    'TONE': 'positive'}]}}

In [56]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- The query asks for the "maximum sales" which is a measure with an adjective "maximum".
- The query specifies the "phone" which is a sub-category under the "Filter" dimension.
- The query mentions the "consumer segment" which is another filter.

Step 2: Map the components to the context
- "Sales" is mapped to the "Sales" entity in the context.
- "Phone" is mapped to the "Phone" entity in the context.
- "Consumer segment" is mapped to the "Consumer" entity in the context.
- "Maximum" is an adjective that can be applied to the "Sales" measure.

Step 3: Create the structured output
- For the "DIMENSION" part, include the "Segment" entity since the query asks for the maximum sales in a segment.
- For the "FILTER" part, include the "Consumer" and "Phone" entities as specified in the query.
- For the "MEASURE" part, include the "Sales" entity with the adjective "maximum" as requested in the query.

Rationale:
The structured output is created

In [57]:
%%time
user_query = 'forecast of sales'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  forecast of sales
CPU times: user 32.2 s, sys: 1.01 s, total: 33.2 s
Wall time: 33.2 s


{'MEASURE': {'sales': [{'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [58]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main measure in the query
The query is asking for a "forecast of sales." From the context, we can identify that the main measure is "Sales."

Step 2: Create a structured output with the main measure
We create a structured output with the main measure "Sales" as the key and an empty list as its value. This list will be used to store any constraints or additional information related to the measure.

Step 3: Identify any constraints or additional information in the query
In this case, there are no specific constraints or additional information mentioned in the query. The query simply asks for a forecast of sales.

Step 4: Add any constraints or additional information to the structured output
Since there are no constraints or additional information, we can leave the list for the "Sales" measure empty.

Step 5: Finalize the structured output
The final structured output contains the main measure "Sales" with an empty list as its value, indicating that there are no specif

In [59]:
%%time
user_query = 'quantity across segments except consumer'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  quantity across segments except consumer
CPU times: user 36.1 s, sys: 1.09 s, total: 37.2 s
Wall time: 37.2 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'consumer': [{'ENTITY': 'Consumer',
    'EXCLUDE': 'True',
    'PARENT': 'Segment'}]},
 'MEASURE': {'quantity': [{'ENTITY': 'Quantity',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [60]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- The query asks for "quantity" which is a measure.
- The query mentions "segments" which is a dimension.
- The query specifies "except consumer" which is a filter.

Step 2: Convert the query into structured output format
- For the "DIMENSION" section, we include "segments" as the main dimension and leave the "RANK" fields empty since there is no ranking specified in the query.
- For the "FILTER" section, we include "consumer" as the entity to be excluded (EXCLUDE: "True") and specify its parent as "Segment".
- For the "MEASURE" section, we include "quantity" as the main measure and leave the "MEASURE CONSTRAINT" fields empty since there is no comparison or constraint specified in the query.

Rationale: The structured output format helps in organizing the query components in a way that can be easily understood and processed by a data analysis system. By breaking down the query into dimensions, filters, and measures, we can ensure that t

In [61]:
%%time
user_query = 'which segment has highest purchase'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  which segment has highest purchase
CPU times: user 42.2 s, sys: 1.04 s, total: 43.2 s
Wall time: 43.2 s


{'DIMENSION': {'segment': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '1'}]}]},
 'MEASURE': {'purchase': [{'ADJECTIVE': ['highest'],
    'ENTITY': 'Purchase Vol',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}],
    'TONE': 'positive'}]}}

In [62]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "which segment has the highest purchase", the main components are "segment" and "highest purchase".

Step 2: Map the components to the context
- "segment" can be mapped to the "Segment" entity under the "DIMENSION" category in the context.
- "highest purchase" can be mapped to the "Purchase Vol" entity under the "MEASURE" category in the context.

Step 3: Determine the required output structure
The output structure should include the "DIMENSION" and "MEASURE" components, as they are the main components of the query.

Step 4: Create the structured output
- For the "DIMENSION" component, include the "Segment" entity and specify the rank as "top 1" since we are looking for the highest purchase.
- For the "MEASURE" component, include the "Purchase Vol" entity and specify the adjective as "highest" to indicate that we are looking for the highest purchase value.

Step 5: Combine the structured output components
Combine the 

In [63]:
%%time
user_query = 'top 2 segments basis discount'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  top 2 segments basis discount
CPU times: user 31.4 s, sys: 1.1 s, total: 32.5 s
Wall time: 32.5 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount'}]}}

In [64]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "top 2 segments basis discount", the main components are "top 2", "segments", and "discount".

Step 2: Map the components to the context
- "segments" can be mapped to the "Segment" entity in the 'DIMENSION' section of the context.
- "discount" can be mapped to the "Discount" entity in the 'MEASURE' section of the context.

Step 3: Create the structured output
- For the "DIMENSION" part of the output, we include the "Segment" entity and specify the ranking information (top 2) as a property of the entity.
- For the "MEASURE" part of the output, we include the "Discount" entity.

Rationale:
The query is asking for the top 2 segments based on the discount. To answer this question, we need to consider the "Segment" dimension and rank them according to the "Discount" measure. The structured output captures this information by including the "Segment" entity with the ranking details (top 2) in the "DIMENSION" section and the 

In [65]:
%%time
user_query = 'binder share of discount by consumer'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  binder share of discount by consumer
CPU times: user 51 s, sys: 1.04 s, total: 52 s
Wall time: 52 s


{'DIMENSION': {'consumer': [{'ENTITY': 'Consumer',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]},
 'DERIVED MEASURE': {'share': [{'ENTITY': 'Why',
    'DERIVED MEASURE CONSTRAINT': [{'DERIVED MEASURE': 'share',
      'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
        'COMPARSION OPERATOR': ''}]}]}]}}

In [66]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- In the query "binder share of discount by consumer", the main components are "binder", "share of discount", and "consumer".

Step 2: Map the components to the context
- "binder" can be mapped to the "Binder" entity under "FILTER".
- "share of discount" can be mapped to the "Why" entity under "DERIVED MEASURE" with the derived measure "share".
- "consumer" can be mapped to the "Consumer" entity under "DIMENSION".

Step 3: Create the structured output
- For the "DIMENSION" section, include the "Consumer" entity since the query is asking for the share of discount by consumers.
- For the "FILTER" section, include the "Binder" entity as the query is asking for the share of discount for binders.
- For the "MEASURE" section, include the "Discount" entity since the query is asking for the share of discount.
- For the "DERIVED MEASURE" section, include the "Why" entity with the derived measure "share" since the query is asking for the share of

In [67]:
%%time
user_query = 'binder and phone share basis sales for corporate'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  binder and phone share basis sales for corporate
CPU times: user 39.8 s, sys: 1.03 s, total: 40.8 s
Wall time: 40.8 s


{'DIMENSION': {'corporate': [{'ENTITY': 'Corporate',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}],
  'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'share': [{'ENTITY': 'Share',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [68]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- The query mentions "binder" and "phone," which are Sub-Category entities.
- It asks for the "share" based on sales for the "corporate" segment.

Step 2: Map the components to the context
- "Binder" and "Phone" are mapped to their respective Sub-Category entities.
- "Corporate" is mapped to the "Corporate" segment.
- "Share" is mapped to the "Share" entity under the "MEASURE" category.

Step 3: Create the structured output
- For the "DIMENSION" section, include the "Corporate" segment as the main dimension.
- For the "FILTER" section, include the "Binder" and "Phone" Sub-Category entities as filters for the main dimension.
- For the "MEASURE" section, include the "Share" entity as the main measure and specify the constraint as being based on sales.

Rationale:
The structured output is created by breaking down the query into its main components and mapping them to the given context. This allows for a clear representation of the query, m

In [69]:
%%time
user_query = 'in corporate, share of phone and binder basis discount'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  in corporate, share of phone and binder basis discount
CPU times: user 48.5 s, sys: 1.1 s, total: 49.6 s
Wall time: 49.6 s


{'DIMENSION': {'corporate': [{'ENTITY': 'Corporate',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}],
  'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]},
 'DERIVED MEASURE': {'share': [{'ENTITY': 'Ratio',
    'DERIVED MEASURE CONSTRAINT': [{'MEASURE': 'Phone',
      'RATIO FILTER': 'Binder'}]}]}}

In [70]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- In the given query, we can identify the following components:
  - Segment: Corporate
  - Sub-Category: Phone and Binder
  - Measure: Discount
  - Derived Measure: Share

Step 2: Map the components to the context
- Using the provided context, we can map the components as follows:
  - Corporate: Segment
  - Phone: Sub-Category
  - Binder: Sub-Category
  - Discount: Measure
  - Share: Derived Measure (Ratio)

Step 3: Create the structured output
- Based on the identified components and their mapping, we can create the structured output as follows:

  1. Add the "DIMENSION" key and include the "corporate" segment.
  2. Add the "FILTER" key and include the "phone" and "binder" sub-categories.
  3. Add the "MEASURE" key and include the "discount" measure.
  4. Add the "DERIVED MEASURE" key and include the "share" derived measure with the "Phone" and "Binder" ratio filter.

Rationale:
- The structured output is created by breaking down the q

In [71]:
%%time
user_query = 'phone to binder ratio of discount in corporate'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  phone to binder ratio of discount in corporate
CPU times: user 53.5 s, sys: 1.04 s, total: 54.6 s
Wall time: 54.6 s


{'DATE': {'': []},
 'DERIVED MEASURE': {'ratio': [{'APPLIED MEASURE': [{'discount': 'Discount'}],
    'DERIVED MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}],
    'ENTITY': 'Ratio',
    'RATIO FILTER': ['Phone', 'Binder']}]},
 'DIMENSION': {'corporate': [{'ENTITY': 'Corporate',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}],
  'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [72]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- Phone and Binder: Sub-Category entities
- Discount: Measure entity
- Corporate: Segment entity
- Ratio: Derived Measure entity

Step 2: Map the components to their respective categories in the structured output
- Phone and Binder are added to the "FILTER" category with their respective parent "Sub-Category".
- Discount is added to the "MEASURE" category.
- Corporate is added to the "DIMENSION" category.
- Ratio is added to the "DERIVED MEASURE" category.

Step 3: Define the relationships between the components
- The query asks for the ratio of sales between Phone and Binder in the Corporate segment.
- In the "DERIVED MEASURE" category, the "ratio" entity is defined with the "APPLIED MEASURE" as "Discount" and the "RATIO FILTER" as "Phone" and "Binder".
- The "DIMENSION" category has the "corporate" entity, which is the context for the ratio calculation.

Step 4: Add any constraints or additional information (if applicable)
- In this c

In [73]:
%%time
user_query = 'segments with discount rate greater than 100k'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  segments with discount rate greater than 100k
CPU times: user 40.9 s, sys: 1.09 s, total: 42 s
Wall time: 42 s


{'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]},
 'MEASURE': {'discount rate': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '100k',
      'COMPARSION OPERATOR': '>'}]}]}}

In [74]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- In the given query "segments with discount rate greater than 100k", the main components are "segments" and "discount rate greater than 100k".

Step 2: Map the components to the context
- "segments" can be mapped to the "Segment" entity under the "DIMENSION" category in the context.
- "discount rate" can be mapped to the "Discount" entity under the "MEASURE" category in the context.

Step 3: Identify the constraints and operators
- In the query, there is a constraint on the discount rate, which is "greater than 100k". This can be represented as a "MEASURE CONSTRAINT" with a "COMPARISON VALUE" of "100k" and a "COMPARISION OPERATOR" of ">".

Step 4: Create the structured output
- Based on the mappings and constraints identified in the previous steps, the structured output is created with the following structure:
  - "DIMENSION" contains the "segments" mapped to the "Segment" entity.
  - "MEASURE" contains the "discount rate" mapped to th

In [75]:
%%time
user_query = 'category with discount rate greater than 10k and quantity less than 2k'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  category with discount rate greater than 10k and quantity less than 2k
CPU times: user 43.1 s, sys: 1.03 s, total: 44.1 s
Wall time: 44.1 s


{'DIMENSION': {'category': [{'ENTITY': 'Sub-Category'}]},
 'MEASURE': {'discount rate': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '10k',
      'COMPARSION OPERATOR': '>'}]}],
  'quantity': [{'ENTITY': 'Quantity',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '2k',
      'COMPARSION OPERATOR': '<'}]}]}}

In [76]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the dimensions and measures in the query.
- In the query, "category" refers to a dimension, which can be mapped to the "Sub-Category" entity in the context.
- "discount rate" refers to a measure, which can be mapped to the "Discount" entity in the context.
- "quantity" refers to another measure, which can be mapped to the "Quantity" entity in the context.

Step 2: Identify the constraints and operators in the query.
- For the "discount rate" measure, the constraint is "greater than 10k", which can be represented as ">" and "10k".
- For the "quantity" measure, the constraint is "less than 2k", which can be represented as "<" and "2k".

Step 3: Structure the output based on the identified dimensions, measures, and constraints.
- Create a "DIMENSION" key and map the "category" to the "Sub-Category" entity.
- Create a "MEASURE" key and map the "discount rate" and "quantity" to their respective entities, "Discount" and "Quantity".
- Add the constraints and operators for eac

In [77]:
%%time
user_query = 'why did discount of phones drop'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  why did discount of phones drop
CPU times: user 51.7 s, sys: 1.06 s, total: 52.8 s
Wall time: 52.8 s


{'DIMENSION': {'phones': [{'ENTITY': 'Phone',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'MEASURE': {'discount': [{'ENTITY': 'Discount',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]},
 'FILTER': {'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}]},
 'DATE VARIABLE': {'order date': [{'ENTITY': 'Order Date',
    'TONE': 'informative'}]},
 'DERIVED MEASURE': {'why': [{'ENTITY': 'Why',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [78]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- In the given query "why did discount of phones drop", the main components are "discount", "phones", and "drop".

Step 2: Map the components to the context
- "discount" can be mapped to the "Discount" entity under the "MEASURE" category.
- "phones" can be mapped to the "Phone" entity under the "FILTER" category.
- "drop" can be inferred as a negative tone and can be represented as a "MEASURE CONSTRAINT" for the "Discount" entity.

Step 3: Create the structured output
- For the "DIMENSION" part, include the "Phone" entity since the query is asking about the phones specifically.
- For the "MEASURE" part, include the "Discount" entity with the negative tone represented as a "MEASURE CONSTRAINT".
- For the "FILTER" part, include the "Corporate" entity as the query doesn't specify any other segment.
- For the "DATE VARIABLE" part, include the "Order Date" entity since there is no mention of any specific date.
- For the "DERIVED MEASURE" par

In [79]:
%%time
user_query = 'correlation of sales and purchase for phone'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  correlation of sales and purchase for phone
CPU times: user 51.4 s, sys: 1.11 s, total: 52.5 s
Wall time: 52.5 s


{'DIMENSION': {'phone': [{'ENTITY': 'Phone',
    'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]},
 'FILTER': {'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}],
  'purchase': [{'ENTITY': 'Purchase',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]},
 'DERIVED MEASURE': {'correlation': [{'ENTITY': 'correlation',
    'APPLIED ON': ['sales', 'purchase']}]}}

In [80]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- The query asks for the correlation between sales and purchase for the "Phone" sub-category.

Step 2: Map the components to the context
- "Phone" is a sub-category, so it falls under the "Sub-Category" dimension.
- "sales" and "purchase" are measures that need to be correlated, so they fall under the "MEASURE" section.
- "correlation" is a derived measure that is applied to the "sales" and "purchase" measures.

Step 3: Create the structured output
- For the "DIMENSION" section, include the "Sub-Category" entity with the "Phone" sub-category as its rank.
- For the "FILTER" section, include the "Phone" sub-category with its parent "Sub-Category".
- For the "MEASURE" section, include the "Sales" and "Purchase" entities with no constraints.
- For the "DERIVED MEASURE" section, include the "correlation" entity with the "sales" and "purchase" measures as its applied on.

Rationale:
- The structured output is designed to provide a clear and o

In [81]:
%%time
user_query = 'what will be the sales in q1 24'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  what will be the sales in q1 24
CPU times: user 36.4 s, sys: 1.11 s, total: 37.5 s
Wall time: 37.5 s


{'DATE VARIABLE': {'q1 24': [{'CONVERTED TIME ELEMENT': 'quarter 1 2024',
    'DATE RANGE': '2024/01/01 - 2024/03/31',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [82]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
The query asks for "sales in q1 24", which means we need to find the sales for the first quarter of 2024.

Step 2: Map the components to the context
- "sales" can be mapped to the "Sales" entity in the MEASURE section of the context.
- "q1 24" can be mapped to the "Order Date" entity in the DATE VARIABLE section of the context.

Step 3: Convert the date reference
- "q1 24" stands for "quarter 1, 2024". We need to convert this to a date range: "2024/01/01 - 2024/03/31".

Step 4: Create the structured output
- In the "DATE VARIABLE" section of the output, we include the converted time element "quarter 1 2024" along with the date range and the corresponding entity "Order Date".
- In the "MEASURE" section of the output, we include the entity "Sales" as the measure we want to retrieve.

Rationale:
The structured output is created by breaking down the query into its main components and mapping them to the context. This allows us to retrieve t

In [83]:
%%time
user_query = 'sales in q1 and q2 2021'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales in q1 and q2 2021
CPU times: user 36.2 s, sys: 1 s, total: 37.2 s
Wall time: 37.2 s


{'DATE VARIABLE': {'q1 and q2 2021': [{'CONVERTED TIME ELEMENT': 'quarter 1 and quarter 2 2021',
    'DATE RANGE': '2021/01/01 - 2021/03/31, 2021/04/01 - 2021/06/30',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [84]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the measure
The query asks for sales, so the measure is identified as "Sales".

Step 2: Identify the date variable and time range
The query asks for sales in the first and second quarters of 2021. To identify the date variable, we use "Order Date" from the context. The time range is from January 1, 2021 to March 31, 2021 for the first quarter and from April 1, 2021 to June 30, 2021 for the second quarter.

Step 3: Create the structured output
The structured output is created by organizing the identified measure and date variable into two main sections: "MEASURE" and "DATE VARIABLE". Under "MEASURE", the identified measure "Sales" is included with its corresponding entity from the context. Under "DATE VARIABLE", the time range identified in Step 2 is included with its corresponding entity from the context, as well as the converted time element and date range.

Rationale:
The query asks for sales data in a specific time range, so it is important to identify the date vari

In [85]:
%%time
user_query = 'how many segments contributing to growth of sales in p3m vs pp'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  how many segments contributing to growth of sales in p3m vs pp
CPU times: user 34.5 s, sys: 1.08 s, total: 35.6 s
Wall time: 35.6 s


{'DATE VARIABLE': {'p3m': 'previous 3 months', 'pp': 'previous period'},
 'DIMENSION': {'segments': 'segments'},
 'MEASURE': {'sales': 'sales'},
 'FILTER': {'none': 'none'}}

In [86]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
- The query asks for the contribution of segments to the growth of sales in two different time periods: P3M (previous 3 months) and PP (previous period).

Step 2: Map the components to the context
- "segments" can be mapped to the "Segment" dimension.
- "sales" can be mapped to the "Sales" measure.
- "P3M" can be mapped to the "Order Date" date variable with the value "previous 3 months".
- "PP" can be mapped to the "Order Date" date variable with the value "previous period".

Step 3: Create the structured output
- For the "DATE VARIABLE" section, include both "P3M" and "PP" with their respective definitions.
- For the "DIMENSION" section, include "Segments" as the dimension being analyzed.
- For the "MEASURE" section, include "Sales" as the measure being analyzed.
- For the "FILTER" section, include "none" as there are no specific filtering requirements in the query.

Rationale:
- The structured output is designed to provide a clear an

In [87]:
%%time
user_query = 'how has sales trended in first week of 2021'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  how has sales trended in first week of 2021
CPU times: user 33.9 s, sys: 1.04 s, total: 34.9 s
Wall time: 34.9 s


{'DATE VARIABLE': {'first week of 2021': [{'CONVERTED TIME ELEMENT': 'first week of 2021',
    'DATE RANGE': '2021/01/01 - 2021/01/07',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [88]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the measure
The query asks for the sales trend, so the measure is identified as "Sales".

Step 2: Identify the date variable and time range
The query specifies the time range as the "first week of 2021". The date variable is identified as "Order Date" from the context. The first week of 2021 is converted to a date range: "2021/01/01 - 2021/01/07".

Step 3: Structure the output
The structured output is organized into two main sections: "DATE VARIABLE" and "MEASURE". Under "DATE VARIABLE", the time range "first week of 2021" is provided along with the corresponding date range and the name of the date entity ("Order Date"). Under "MEASURE", the identified measure "sales" is listed along with its corresponding entity ("Sales").

Rationale:
The query asks for the sales trend during a specific time range, so the main focus is on identifying the measure (sales) and the date variable with the corresponding time range (first week of 2021). The structured output is organized in 

In [89]:
%%time
user_query = 'what will be sales in 1st 5 days of 2024'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  what will be sales in 1st 5 days of 2024
CPU times: user 48.9 s, sys: 1.05 s, total: 49.9 s
Wall time: 49.9 s


{'DATE VARIABLE': {'1st 5 days of 2024': [{'CONVERTED TIME ELEMENT': 'first 5 days of january 2024',
    'DATE RANGE': '2024/01/01 - 2024/01/05',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [90]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "what will be sales in 1st 5 days of 2024", the main components are "sales" and "1st 5 days of 2024".

Step 2: Map the components to the context
- "sales" can be mapped to the "Sales" entity under the "MEASURE" category in the context.
- "1st 5 days of 2024" can be mapped to the "Order Date" entity under the "DATE VARIABLE" category in the context.

Step 3: Convert the components into structured output format
- For "sales", create a dictionary with the key "sales" and the value as a list containing a dictionary with the key "ENTITY" and the value "Sales".
- For "1st 5 days of 2024", create a dictionary with the key "1st 5 days of 2024" and the value as a list containing a dictionary with the keys "CONVERTED TIME ELEMENT", "DATE RANGE", and "ENTITY" and their respective values "first 5 days of january 2024", "2024/01/01 - 2024/01/05", and "Order Date".

Step 4: Combine the structured output dictionaries
- Combine the d

In [91]:
%%time
user_query = 'sales and purchase across yoy'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales and purchase across yoy
CPU times: user 30.5 s, sys: 1.04 s, total: 31.5 s
Wall time: 31.5 s


{'DATE VARIABLE': {'yoy': [{'CONVERTED TIME ELEMENT': 'year over year',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol'}],
  'sales': [{'ENTITY': 'Sales'}]}}

In [92]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the measures
The query mentions "sales" and "purchase", which are measures. From the context, we can map "sales" to the "Sales" entity and "purchase" to the "Purchase Vol" entity.

Step 2: Identify the date variable
The query mentions "yoy", which stands for "year over year". From the context, we can map this to the "Order Date" entity with the date reference of "yoy".

Step 3: Structure the output
The structured output should have two main sections: "MEASURE" and "DATE VARIABLE". In the "MEASURE" section, we include the identified measures "sales" and "purchase" with their respective entities. In the "DATE VARIABLE" section, we include the identified date reference "yoy" with the corresponding entity "Order Date".

Rationale: The query is asking for sales and purchase data across different years. By identifying the measures and date variable, we can provide a structured output that includes the relevant information in a clear and organized manner.

In summary, the str

In [93]:
%%time
user_query = 'sales and purchase during last one year'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  sales and purchase during last one year
CPU times: user 53.5 s, sys: 1.09 s, total: 54.6 s
Wall time: 54.6 s


{'DATE VARIABLE': {'last one year': [{'CONVERTED TIME ELEMENT': 'last one year',
    'DATE RANGE': '2020/09/01 - 2021/08/31',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol'}],
  'sales': [{'ENTITY': 'Sales'}]}}

In [94]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the measures
The query mentions "sales" and "purchase", which are measures. We map these to the "Sales" and "Purchase Vol" entities in the context.

Step 2: Identify the date variable
The query mentions "last one year", which is a time expression. We map this to the "Order Date" entity in the context and calculate the date range for "last one year".

Step 3: Structure the output
We create a structured output with two main sections: "MEASURE" and "DATE VARIABLE". Under "MEASURE", we include the "Sales" and "Purchase Vol" entities. Under "DATE VARIABLE", we include the "last one year" time expression and its corresponding date range.

Rationale:
The structured output provides a clear and organized representation of the query, making it easier to understand and process. By identifying and mapping the measures and date variables, we can provide a detailed analysis of the sales and purchase data for the last one year.


























































In [23]:
%%time
user_query = 'In q1 2023, what was sales'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  In q1 2023, what was sales
CPU times: user 30.2 s, sys: 1.04 s, total: 31.2 s
Wall time: 31.2 s


{'DATE VARIABLE': {'q1 2023': [{'CONVERTED TIME ELEMENT': 'quarter 1 2023',
    'DATE RANGE': '2023/01/01 - 2023/03/31',
    'ENTITY': 'Order Date'}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [24]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "In q1 2023, what was sales", the main components are "q1 2023" (date range) and "sales" (measure).

Step 2: Map the components to the context
- "q1 2023" can be mapped to the "Order Date" entity under "DATE VARIABLE" in the context.
- "sales" can be mapped to the "Sales" entity under "MEASURE" in the context.

Step 3: Create the structured output
- For the "DATE VARIABLE" component, include the date range (q1 2023) and the corresponding entity ("Order Date").
- For the "MEASURE" component, include the entity ("Sales").

Rationale:
The structured output is created by identifying the main components of the query and mapping them to the given context. This helps in understanding the user's intent and provides a structured format that can be used for further analysis or processing.

By following these steps, the given query "In q1 2023, what was sales" is converted into a structured output that includes the date range an

In [25]:
%%time
user_query = 'What are drivers of sales growth'
print('user query: ', user_query)
output, raw = predict_template_query_v1(user_query=user_query)
eval(output)

user query:  What are drivers of sales growth
CPU times: user 35.1 s, sys: 1.03 s, total: 36.1 s
Wall time: 36.2 s


{'DIMENSION': {'drivers': [{'ENTITY': 'Key Drivers',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '1'}]}]},
 'MEASURE': {'sales growth': [{'ENTITY': 'Sales',
    'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '',
      'COMPARSION OPERATOR': ''}]}]}}

In [26]:
print('Step 1:' +raw.split('\nStep 1:')[1])

Step 1: Identify the main components of the query
In the given query "What are drivers of sales growth", the main components are "drivers" and "sales growth".

Step 2: Map the components to the context
- "drivers" can be mapped to the "Key Drivers" dimension under "DIMENSION".
- "sales growth" can be mapped to the "Sales" measure under "MEASURE".

Step 3: Create the structured output
- For the "DIMENSION" part, we include the "Key Drivers" dimension and specify the rank as "top 1" since we are asking for the main drivers.
- For the "MEASURE" part, we include the "Sales" measure and leave the measure constraint as it is since we are asking for the drivers of sales growth.

Rationale:
The query is asking for the main drivers of sales growth. To answer this, we need to identify the key drivers that have the most impact on sales. This can be done by analyzing various dimensions and their contribution to sales growth. The structured output is designed to capture this information by includin

In [35]:
raw

'<s> Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile pho